## Import Functions and Spark Config

In [1]:
%run ./read_file.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 18:53:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/27 18:53:11 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/02/27 18:53:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Set Environment

In [4]:
# env - dev / prod
# size - small / medium / large
env='prod'
size='medium'

## Data Processing

In [5]:
df_ecarbro = readCheckpoint('ecar-bro', env, size).cache()

--- 4.473739147186279 seconds ---


In [ ]:
df_ecar = readCheckpoint('ecar', env, size).cache()

In [ ]:
df_ecarbro = df_ecarbro.join(df_ecar, df_ecarbro["id"] ==  df_ecar["id"], "inner").cache()
df_ecar.unpersist()
df_ecarbro.unpersist()

In [ ]:
df_ecarbro

In [ ]:
df_ecarbro.limit(2).show()

In [ ]:
df_labels = readCheckpoint('labels', env, size='large')

In [ ]:
df_labels.count()

In [ ]:
df_labels.limit(5).toPandas()

In [ ]:
df_labels.toPandas().groupby('hostname')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.toPandas().groupby('action')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.toPandas().groupby('object')['id'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
df_labels.distinct()